In [1]:
import os
import argparse
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from utils import preprocess_data, SequenceDataset, train_model, score_model, log, get_predictions, plot_predictions
from model import LSTMRegression

/Users/twitter/anaconda3/envs/py310_ts/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


```
KEW GARDENS_ent        16800
15 ST-PROSPECT_ex      16800
KEW GARDENS_ex         16800
161/YANKEE STAD_ex     16800
161/YANKEE STAD_ent    16800
15 ST-PROSPECT_ent     16800
```

In [2]:
df = pd.read_parquet("../data/mta_subway_221231_100wk_dbscan.parquet")
df = df.drop(
    [
        "KEW GARDENS_ent", 
        "KEW GARDENS_ex", 
        "15 ST-PROSPECT_ent", 
        "15 ST-PROSPECT_ex", 
        "161/YANKEE STAD_ent",
        "161/YANKEE STAD_ex"
        ],
    axis=1)
df = df.fillna(0)
df.shape

(16800, 752)

In [3]:
df.sum()==0

1 AV_ent             False
1 AV_ex              False
103 ST-CORONA_ent    False
103 ST-CORONA_ex     False
103 ST_ent           False
                     ...  
WTC-CORTLANDT_ex     False
YORK ST_ent          False
YORK ST_ex           False
ZEREGA AV_ent        False
ZEREGA AV_ex         False
Length: 752, dtype: bool

In [4]:
df.head()

,1 AV_ent,1 AV_ex,103 ST-CORONA_ent,103 ST-CORONA_ex,103 ST_ent,103 ST_ex,104 ST_ent,104 ST_ex,110 ST_ent,110 ST_ex,...,WOODLAWN_ent,WOODLAWN_ex,WORLD TRADE CTR_ent,WORLD TRADE CTR_ex,WTC-CORTLANDT_ent,WTC-CORTLANDT_ex,YORK ST_ent,YORK ST_ex,ZEREGA AV_ent,ZEREGA AV_ex
TIME,,,,,,,,,,,,,,,,,,,,,
2021-01-23 00:00:00,12.0,63.5,233.25,37.5,12.0,31.5,31.5,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.0,5.75,0.0,0.0,0.25,6.0
2021-01-23 01:00:00,12.0,63.5,233.25,37.5,12.0,31.5,31.5,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.0,5.75,0.0,0.0,0.25,6.0
2021-01-23 02:00:00,12.0,63.5,233.25,37.5,12.0,31.5,31.5,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.0,5.75,0.0,0.0,0.25,6.0
2021-01-23 03:00:00,12.0,63.5,233.25,37.5,12.0,31.5,31.5,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.0,5.75,0.0,0.0,0.25,6.0
2021-01-23 04:00:00,12.0,63.5,233.25,37.5,12.0,31.5,31.5,3.25,31.25,27.75,...,40.0,12.25,1.75,2.5,1.0,5.75,0.0,0.0,0.25,6.0


In [5]:
forecast_lead=0
batch_size=32
sequence_length=30
learning_rate = 5e-5
num_hidden_units=16
num_layers=1
dropout=0
num_epochs=2

In [6]:
batch_size

32

In [7]:
df_train, df_test, features = preprocess_data(
        df,
        #target_feature, 
        forecast_lead=forecast_lead,
        train_test_split=0.8
        )

In [ ]:
df_train.head(2)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.mean()

In [ ]:
df_test.mean()

In [ ]:
df_train['1 AV_ent'].plot()

In [ ]:
df_test['1 AV_ent'].plot()

In [ ]:
#df_train.isna().sum().sort_values(ascending=False).head(20)

In [8]:
train_dataset = SequenceDataset(
    df_train,
    #target=None,
    features=features,
    sequence_length=sequence_length,
    forecast_lead=1
    )

test_dataset = SequenceDataset(
    df_test,
    #target=None,
    features=features,
    sequence_length=sequence_length,
    forecast_lead=1
    )

In [9]:
forecast_lead = 1
df_train[['1 AV_ent']].iloc[:-forecast_lead,:] # all but the last 'forecast_lead' samples so that there is a y associated

,1 AV_ent
TIME,
2021-01-23 00:00:00,-1.290736
2021-01-23 01:00:00,-1.290736
2021-01-23 02:00:00,-1.290736
2021-01-23 03:00:00,-1.290736
2021-01-23 04:00:00,-1.290736
...,...
2022-08-05 18:00:00,1.471583
2022-08-05 19:00:00,1.471583
2022-08-05 20:00:00,1.471583


In [ ]:
df_train[['1 AV_ent']].iloc[1,:]

In [ ]:
test_dataset.__len__()

In [ ]:
train_dataset.__getitem__(25)[0].shape

In [ ]:
train_dataset.__getitem__(25)[1].mean()

In [ ]:
test_dataset.__getitem__(25)[1].mean()

In [ ]:
batch_size

In [10]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_dataset.__len__()

In [ ]:
#for X, y in train_loader:
#    print(X.shape)
#    print(y.shape)
#    print("")

In [ ]:
y

In [ ]:
X.shape

In [ ]:
df.shape

In [ ]:
y.shape

In [11]:
class LSTMRegression(nn.Module):
    def __init__(self, num_features, hidden_units, num_layers=1,dropout=0):
        super().__init__()
        self.num_features = num_features
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers,
            dropout=self.dropout
        )
        self.linear = nn.Linear(in_features=self.hidden_units, out_features=self.num_features)
        
    def forward(self,x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        _,(hn,_) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0])#.flatten()
        return out
        
model = LSTMRegression(
        num_features=len(features), 
        hidden_units=num_hidden_units,
        num_layers=num_layers,
        dropout=dropout
        )#.to(device=device)

In [14]:
def train_model(data_loader, model, loss_function, optimizer, device=torch.device("mps")):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        #X = X.to(device=device)
        #y = y.to(device=device)
        output = model(X)
        loss = loss_function(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    avg_loss = total_loss/num_batches
    #print(f"Train loss: {avg_loss}")
    return avg_loss

In [15]:
model = LSTMRegression(
        num_features=len(features), 
        hidden_units=num_hidden_units,
        num_layers=num_layers,
        dropout=dropout
        )#.to(device=device)

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

avg_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)

In [16]:
avg_loss

1.00462486743927

In [17]:
test_score = score_model(test_loader, model, loss_function)

In [18]:
test_score

1576.0388508785338

In [19]:
loss_function

MSELoss()

In [21]:
y_star = model(X)
y_star.shape

NameError: name 'X' is not defined

In [ ]:
loss_function(y_star, y).item()

In [ ]:
y_star